# 미니 의도분석기 실습

## 학습목차

0. 필요 라이브러리 임포트
2. 사전 데이터 확인
 - 개체명 파일, 액션-개체명 파일, 액션-패턴 파일
2. 발화 형태소 분석 (Komoran 형태소 분석기 사용)
3. 분석된 형태소 중 명사, 숫자에 한하여 개체명 검색
 - 일치하는 개체명이 있을 시 치환 (ex. 에어컨 -> iot_device)
4. 분석된 발화를 액션-패턴 파일 내용과 비교검사
5. 입력된 발화와 패턴이 일치하는 액션 찾기
 - 발견 시 액션 반환, 발견하지 못하면 None 반환


# 0. 필요 라이브러리 임포트

### Python, konlpy

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install konlpy

### Pandas, Komoran

In [ ]:
import pandas as pd
from konlpy.tag import Komoran

# 1. 사전 데이터 확인

### <font color=green> 구글 드라이브 연동 </font>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/My Drive"
!git clone "https://github.com/friendly036/test.git"

/content/drive/My Drive
Cloning into 'test'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 29 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), 212.84 KiB | 525.00 KiB/s, done.


In [ ]:
%cd "/content/drive/My Drive/test"
%ls

/content/drive/My Drive/test
Action_NE_Table.txt
db_KOMORAN.txt
home_device.txt
mini_nlp.ipynb
mini_nlp_ipynb의_사본.ipynb
ne_value.csv
ne_value.xls
Python활용AI모델링_01_전처리_정답_v3_20220219.ipynb
Python활용AI모델링_01_전처리_퀴즈_easy_v3_20220219.ipynb
test.txt
Welcome.ipynb


### <font color=green>개체명 파일 : ne_value.csv</font>

In [ ]:
ne_value = pd.read_csv('ne_value.csv', encoding='utf-8')

In [ ]:
ne_value.head()

,ne,value
0,Iot_device,에어컨
1,lc_city,서울
2,lc_city,부산
3,dt_day,오늘
4,dt_day,내일


### <font color=green>액션-개체명 파일 : action_ne.csv</font>

In [ ]:
ne_value = pd.read_csv('ne_value.csv', encoding='utf-8')

### <font color=green> 액션-패턴 파일 : action_pattern.csv </font>

# 2. 발화 형태소 분석

In [ ]:
komoran = Komoran()
komoran.tagset

{'EC': '연결 어미',
 'EF': '종결 어미',
 'EP': '선어말어미',
 'ETM': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JKB': '부사격 조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JKV': '호격 조사',
 'JX': '보조사',
 'MAG': '일반 부사',
 'MAJ': '접속 부사',
 'MM': '관형사',
 'NA': '분석불능범주',
 'NF': '명사추정범주',
 'NNB': '의존 명사',
 'NNG': '일반 명사',
 'NNP': '고유 명사',
 'NP': '대명사',
 'NR': '수사',
 'NV': '용언추정범주',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SH': '한자',
 'SL': '외국어',
 'SN': '숫자',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'VA': '형용사',
 'VCN': '부정 지정사',
 'VCP': '긍정 지정사',
 'VV': '동사',
 'VX': '보조 용언',
 'XPN': '체언 접두사',
 'XR': '어근',
 'XSA': '형용사 파생 접미사',
 'XSN': '명사파생 접미사',
 'XSV': '동사 파생 접미사'}

In [ ]:
komoran = Komoran()

# 패턴 파일 읽기
db = pd.read_table('db_KOMORAN.txt', sep = "\t", engine='python', encoding = "utf-8")
db = pd.Series(db.main_action.values, index=db.corpus_pattern).to_dict()

action_ne_f = open("Action_NE_Table.txt", 'r', encoding="UTF-8")
lines = action_ne_f.readlines()
action_ne_f.close()
action_table_NE = []

# 액션테이블 읽기
for line in lines:
    items = line.strip().split(':\t')
    if len(items) == 2:
        action_table_NE.append([items[0], items[1]]) # [action , entity]
    else:
        action_table_NE.append([items[0][:-1], ''])

# 엔티티 파일 읽기
entity_dict = {}    # {"home_device":[{"device_status":[에어컨, 에어컨디셔너, ...]}]}
for action, entity_files in action_table_NE:
    if entity_files:
        for entity_file in entity_files.split(", "):
            if not entity_file in entity_dict:
                f = open(entity_file+".txt", 'r', encoding="UTF-8")
                ett_lines = f.readlines()
                ett_lines = list(map(lambda s: s.strip(), ett_lines))
                ett_name_dic = {}  # {"device_status":[에어컨:aircon, 에어컨디셔너:aircon, 벽걸이에어컨:aircon...]}
                ett_name = ''
                etts = []
                for line in ett_lines:
                    if line[0:4] == "<ne>":
                        if etts:
                            ett_name_dic[ett_name] = etts
                            etts = []
                        ett_name = line.split('\t')[1]
                    else:
                        if line and not line.startswith('#'):
                            etts.append(line.strip())
                
                if etts:
                    ett_name_dic[ett_name] = etts        
                entity_dict[entity_file] = ett_name_dic
                f.close()

    else:
        entity_dict[entity_files] = {}

# print(entity_dict.keys())

def detect_action(utterance):
    output_dict = {"action":'None', "pos":'', "matched pattern":'None', "detected_NE":defaultdict(list), "all_utterances":[]}
    pos_list = komoran.pos(utterance)
    output_dict["pos"] = pos_list
    parsed_utters = ''  # 각 메인액션별 발화 분석 결과
    parsed_patterns = []

    for action, entity_files in action_table_NE: # 엔티티 치환은 이 루프(=액션 수)만큼
        parsed_words = []
        for pos_tup in pos_list:    # 각 단어별로
            is_replaced = False
            for entity_file in entity_files.split(", "): # 엔티티 일치 검사
                if pos_tup[1] in ["NNG", "NNP", "NR", "SL", "SN"]:            
                    for entity_name in entity_dict[entity_file].keys(): # [<ne> home_device, device_status, device_mode, ...]
                        for entity in entity_dict[entity_file][entity_name]:    # [에어컨, 에어컨디셔너, ...]
                            if pos_tup[0] == entity.split(":")[0]:
                                parsed_words.append(entity_name)
                                is_replaced = True
                                output_dict["detected_NE"][action].append(entity)
                                break
                        if is_replaced:
                            break
            if not is_replaced and pos_tup[1] in ["NNG", "NNP", "NR", "VV", "VA", "MM", "MAG", "SL", "XR", "SN"]:   # 명/동/ 형용사일 때만 인식
                parsed_words.append(pos_tup[0])
            elif pos_tup == ('말', 'VX'):
                parsed_words.append(pos_tup[0])
            elif pos_tup == ('꺼', 'NNB'):
                parsed_words.append(pos_tup[0])                

        parsed_utters += action+": ["+', '.join(parsed_words)+"]\r\n"
        # parsed_utters.append(action+": ["+', '.join(parsed_words)+"]")
        parsed_patterns.append(' '.join(parsed_words))
    
    output_dict["all_utterances"] = parsed_utters

    # 패턴 매칭
    for parsed_pattern in parsed_patterns:
        for pattern, action in db.items():
            if pattern == parsed_pattern:
                output_dict["action"] = action
                output_dict["matched pattern"] = pattern
                return output_dict

    # 여기 들어오면 액션은 None
        
    ut = parsed_pattern
    reference_sequences = [["tv_search_title", ["나의 해방일지", "꼬리에 꼬리를 무는 이야기", "찾 찾아보 틀 켜"]], ["tv_play",["번 채널 번 틀"]], ["tv_volume_up", ["볼륨 소리 음량 키우 늘리 크 올리 높이"]], ["tv_volume_down", ["볼륨 소리 음량 줄이 낮 낮추 작 적"]], ["sh_device_power_on", ["켜 틀 돌리 작동"]], ["sh_device_power_off", ["끄 꺼 종료 정지"]], ["wt_search_weather", ["날씨 기상예보 일기예보"]]]    # 콜랩에 올린 결과

    bleu_dict = {"tv_search_title":0, "tv_play":0, "tv_volume_up":0, "tv_volume_down":0, "sh_device_power_on":0, "sh_device_power_off":0, "wt_search_weather":0}
    for reference_sequence in reference_sequences:
        bleu_dict[reference_sequence[0]] = sentence_bleu(reference_sequence[1], ut, weights=(3/6, 2/6, 1/6))
        if output_dict["detected_NE"][reference_sequence[0]]:
            bleu_dict[reference_sequence[0]] *= 3*len(output_dict["detected_NE"][reference_sequence[0]])

    bleu_dict = sorted(bleu_dict.items(), key=lambda x:x[1], reverse=True)
    output_dict["action"] = bleu_dict[0][0]

    return output_dict


In [ ]:
detect_action("부산 날씨")

{'action': 'wt_search_weather',
 'pos': [('부산', 'NNP'), ('날씨', 'NNP')],
 'matched pattern': 'location 날씨',
 'detected_NE': defaultdict(list,
             {'sh_device_power_on': ['부산'],
              'sh_device_power_off': ['부산'],
              'wt_search_weather': ['부산']}),
 'all_utterances': 'tv_volume_up: [부산, 날씨]\r\ntv_volume_down: [부산, 날씨]\r\nsh_device_power_on: [location, 날씨]\r\nsh_device_power_off: [location, 날씨]\r\nwt_search_weather: [location, 날씨]\r\n'}